### Import modules and initialize

In [1]:
# import required modules
import ee
import folium
import shapefile
import numpy as np

In [2]:
# initialize earth engine api
ee.Initialize()

### Collect imagery

In [75]:
# function to convert shapefile into feature collection
def convert_shp2feature (shp_fp):
    shp = shapefile.Reader(shp_fp)
    fields = shp.fields[1:]
    field_names = [field[0] for field in fields]
    wgs84 = ee.Projection('EPSG:4326')
    features = []
    for sr in shp.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        ee_geometry = ee.Geometry(geom,'EPSG:4326')
        feat = ee.Feature(ee_geometry, atr)
        features.append(feat)
    return ee.FeatureCollection(features)

# function to cloud mask sentinel imagery
def mask_clouds_sentinel (image):
    qa = image.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear_pixels = qa.bitwiseAnd(cloudBitMask).eq(0)\
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(clear_pixels).divide(10000)

In [76]:
test_feat = convert_shp2feature('../../data/ghana/tiles/ghana_l_11/ghana_l_11.shp')
bounds = ee.Feature(test_feat.first()).geometry()

bounds.coordinates().getInfo()[0]

[[-0.01, 6.99], [-0.01, 8.01], [1.01, 8.01], [1.01, 6.99], [-0.01, 6.99]]

In [77]:
xmax = bounds.coordinates().getInfo()[0][2][0]
xmin = bounds.coordinates().getInfo()[0][0][0]
ymin = bounds.coordinates().getInfo()[0][3][1]
ymax = bounds.coordinates().getInfo()[0][1][1]

xmean = np.average([xmin, xmax])
ymean = np.average([ymin, ymax])

poi = [ymean, xmean]
centroid = ee.Geometry.Point([xmean, ymean])

In [78]:
image = ee.ImageCollection('COPERNICUS/S2_SR')\
    .map(mask_clouds_sentinel)\
    .filterBounds(centroid)\
    .median()\
    .clip(bounds)
#

### Display

In [89]:
# function to display ee image tiles on folium map
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+"/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

In [90]:
# create map
folium.Map.add_ee_layer = add_ee_layer
eeMap = folium.Map(location=poi, zoom_start=15)

In [91]:
# define visualization parameters
centroid

In [92]:
# add layers to map
eeMap.add_ee_layer(image, {'bands':['B4','B3','B2'], 'min':0, 'max':0.2}, 'precipitation')

In [93]:
# display map
eeMap.add_child(folium.LayerControl())
folium.Marker(poi).add_to(eeMap)
display(eeMap)